# 2D Coordinate transformation

## Example 3: 2DOF robot arm - Forward kinematics  

In [1]:
from matplotlib import pyplot as plt
from numpy.linalg import inv
from IPython import display
import numpy as np
import math

PI = 3.1415926535

def plot3x3Mat(m):
    print("[%6.2f\t%6.2f\t%6.2f]"%(m[0,0],m[0,1],m[0,2]))
    print("[%6.2f\t%6.2f\t%6.2f]"%(m[1,0],m[1,1],m[1,2]))
    print("[%6.2f\t%6.2f\t%6.2f]"%(m[2,0],m[2,1],m[2,2]))
    
def IIDrotM(phi):
    m = np.array([[math.cos(float(phi)),math.sin(float(phi)),0],[-math.sin(float(phi)),math.cos(float(phi)),0],[0,0,1]])
    return m

def IIDtraM(v):
    m = np.array([[1,0,float(v[0])],[0,1,float(v[1])],[0,0,1]])
    return m

def IIDtraHTMvec(x,y):
    v = np.array([ [float(x)],[float(y)],[1] ])
    return v

### (I) Scenario

In [2]:
# definition of parameters of the kinematic chain
l0 = 1
l1 = 1
l2 = 1
phi0 = 0
phi1 = 0

### (II) Derive the three HTMs  $~~~^{0}_{1}T~~~$, $~~~^{1}_{2}T~~~$ and $~~~^{2}_{3}T~~~$ 

#### (II).a  HTM $~~~^{0}_{1}T~~~$

In [3]:
TR_1_0 = IIDtraM(IIDtraHTMvec(0,l0))
R_1_0  = IIDrotM(-phi0)

T_1_0 = TR_1_0 @ R_1_0

In [4]:
#Output final matrix TR_1_0
plot3x3Mat(T_1_0)

[  1.00	  0.00	  0.00]
[  0.00	  1.00	  1.00]
[  0.00	  0.00	  1.00]


#### (II).b  HTM $~~~^{1}_{2}T~~~$

In [5]:
TR_2_1 = IIDtraM(IIDtraHTMvec(l1,0))
R_2_1  = IIDrotM(-phi1)

T_2_1 = TR_2_1 @ R_2_1

In [6]:
#Output final matrix TR_2_1
plot3x3Mat(T_2_1)

[  1.00	  0.00	  1.00]
[  0.00	  1.00	  0.00]
[  0.00	  0.00	  1.00]


#### (II).c  HTM $~~~^{2}_{3}T~~~$

In [7]:
TR_3_2 = IIDtraM(IIDtraHTMvec(l2,0))
R_3_2  = IIDrotM(0)

T_3_2 = TR_3_2 @ R_3_2

In [8]:
#Output final matrix TR_2_1
plot3x3Mat(T_3_2)

[  1.00	  0.00	  1.00]
[  0.00	  1.00	  0.00]
[  0.00	  0.00	  1.00]


### (III) Define a function for the final parameterized HTM $~~~^{0}_{3}T(\phi_0, \phi_1, ...)~~~$

In [9]:
def FK(PHI0,PHI1, L0,L1,L2):
    htm_1_0 = IIDtraM(IIDtraHTMvec(0,L0)) @ IIDrotM(-PHI0)
    htm_2_1 = IIDtraM(IIDtraHTMvec(L1,0)) @ IIDrotM(-PHI1)
    htm_3_2 = IIDtraM(IIDtraHTMvec(L2,0)) @ IIDrotM(0)
    htm_3_0 = htm_1_0 @ htm_2_1 @ htm_3_2
    return htm_3_0

### (VI) Calculate position of the TCP for  given joint positions $\phi_0$ and $\phi_1$

In [10]:
phi0 = 0
phi1 = 0

In [11]:
TCP = FK(phi0, phi1,l0,l1,l2)

In [12]:
plot3x3Mat(TCP)

[  1.00	  0.00	  2.00]
[  0.00	  1.00	  1.00]
[  0.00	  0.00	  1.00]


In [13]:
print("x_TCP = ",end=""); print("%6.2f"%TCP[0,2])
print("y_TCP = ",end=""); print("%6.2f"%TCP[1,2])

x_TCP =   2.00
y_TCP =   1.00


### (V) Check for plausibility

The result seems plausible because *\<PUT YOUR ANSWER HERE...\>*